In [14]:
!pip install pygame

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.6 MB 2.4 MB/s eta 0:00:05
   ---- ----------------------------------- 1.3/10.6 MB 3.4 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/10.6 MB 4.3 MB/s eta 0:00:02
   ------------------ --------------------- 5.0/10.6 MB 5.9 MB/s eta 0:00:01
   ----------------------------- ---------- 7.9/10.6 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------  10.5/10.6 MB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 8.5 MB/s eta 0:00:00


In [1]:
import pygame, sys, random
from pygame.locals import *

# Initialize Pygame
pygame.init()

# Grid and display settings
CELL_SIZE = 5
GRID_WIDTH = 100
GRID_HEIGHT = 100

SCREEN_WIDTH = GRID_WIDTH * CELL_SIZE
SCREEN_HEIGHT = GRID_HEIGHT * CELL_SIZE

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Langton's Ant with Pheromone Influence")
clock = pygame.time.Clock()
FPS = 60

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED   = (255, 0, 0)
BLUE  = (0, 0, 255)
GRAY  = (200, 200, 200)

# Directions: 0=up, 1=right, 2=down, 3=left
dx = [-1, 0, 1,  0]
dy = [ 0, 1, 0, -1]

# Pheromone influence parameters:
SELF_STRAIGHT = 0.8   # If own pheromone, 80% chance to go straight
CROSS_STRAIGHT = 0.2  # If cross pheromone, 20% chance to go straight
PHEROMONE_LIFETIME = 5  # After 5 moves, pheromone decays

# Create grid: each cell holds:
#  'color': 0 for white, 1 for black
#  'pheromone': None or (ant_id, age)
def create_grid(width, height):
    grid = [[{'color': 0, 'pheromone': None} for _ in range(width)] for _ in range(height)]
    return grid

grid = create_grid(GRID_WIDTH, GRID_HEIGHT)

# Define an Ant class
class Ant:
    def __init__(self, ant_id, x, y, direction):
        self.id = ant_id
        self.x = x
        self.y = y
        self.dir = direction

    def update(self):
        # Determine cell properties
        cell = grid[self.x][self.y]
        base_turn = None
        
        # Standard rule: if white -> turn right; if black -> turn left
        if cell['color'] == 0:
            base_turn = 1  # right turn: +1 mod 4
        else:
            base_turn = -1  # left turn: -1 mod 4
        
        # Check pheromone in the cell:
        if cell['pheromone'] is not None:
            pher_ant, age = cell['pheromone']
            # Influence decays linearly: full effect if age=0, no effect if age>=PHEROMONE_LIFETIME
            effect = max(0, 1 - age / PHEROMONE_LIFETIME)
            if pher_ant == self.id:
                # Self-pheromone: probability to go straight is SELF_STRAIGHT * effect + (1 - effect)*0
                p_straight = SELF_STRAIGHT * effect
            else:
                # Cross-pheromone: probability to go straight is CROSS_STRAIGHT * effect
                p_straight = CROSS_STRAIGHT * effect
        else:
            p_straight = 0
        
        # Decide whether to override turning rule based on pheromone influence
        if random.random() < p_straight:
            # Move straight; no turning change
            new_dir = self.dir
        else:
            # Follow standard rule
            new_dir = (self.dir + base_turn) % 4
        
        self.dir = new_dir

        # Flip the cell color (standard Langton's ant rule)
        grid[self.x][self.y]['color'] = 1 - grid[self.x][self.y]['color']
        
        # Deposit new pheromone: per replacement rule, new pheromone overrides any old one.
        grid[self.x][self.y]['pheromone'] = (self.id, 0)
        
        # Move forward one cell according to new direction
        self.x = (self.x + dx[self.dir]) % GRID_HEIGHT
        self.y = (self.y + dy[self.dir]) % GRID_WIDTH

# Initialize two ants with different IDs and colors
x1=random.randint(GRID_HEIGHT//4,3*GRID_HEIGHT//4)
x2=random.randint(10,GRID_HEIGHT//2)
y1=random.randint(GRID_WIDTH//4,3*GRID_WIDTH//4)
y2=random.randint(10,GRID_WIDTH//2)
ants = [
    Ant(1,x1,y1, 0),      # Start near center, arbitrary direction
    Ant(2,x1+10,y1+10, 2)     # A bit offset from the first ant
]

# Function to update pheromones in the entire grid (decay pheromones)
def update_pheromones():
    for i in range(GRID_HEIGHT):
        for j in range(GRID_WIDTH):
            cell = grid[i][j]
            if cell['pheromone'] is not None:
                ant_id, age = cell['pheromone']
                age += 1
                if age >= PHEROMONE_LIFETIME:
                    cell['pheromone'] = None
                else:
                    cell['pheromone'] = (ant_id, age)

# Main simulation loop
running = True
while running:
    clock.tick(FPS)
    
    for event in pygame.event.get():
        if event.type == QUIT:
            running = False

    # Update each ant
    for ant in ants:
        ant.update()
    
    # Decay pheromones in grid
    update_pheromones()
    
    # Draw the grid
    screen.fill(WHITE)
    for i in range(GRID_HEIGHT):
        for j in range(GRID_WIDTH):
            cell = grid[i][j]
            rect = pygame.Rect(j*CELL_SIZE, i*CELL_SIZE, CELL_SIZE, CELL_SIZE)
            # Base color: white for 0, black for 1
            color = WHITE if cell['color'] == 0 else BLACK
            
            # If there's a pheromone, tint the cell
            if cell['pheromone'] is not None:
                ant_id, age = cell['pheromone']
                # Compute intensity factor (full intensity at age 0, decays linearly)
                intensity = 1 - (age / PHEROMONE_LIFETIME)
                if ant_id == 1:
                    # Tint red
                    color = (int(255 * intensity), 0, 0)
                else:
                    # Tint blue
                    color = (0, 0, int(255 * intensity))
            
            pygame.draw.rect(screen, color, rect)
    
    # Draw ants as bright squares
    for ant in ants:
        rect = pygame.Rect(ant.y*CELL_SIZE, ant.x*CELL_SIZE, CELL_SIZE, CELL_SIZE)
        if ant.id == 1:
            pygame.draw.rect(screen, (255,255,0), rect)  # Yellow for ant 1
        else:
            pygame.draw.rect(screen, (0,255,255), rect)  # Cyan for ant 2

    pygame.display.flip()

pygame.quit()
sys.exit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Downloads\Anaconda\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
